In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt
import pandas as pd

In [2]:
# data_generation_process = "SPO_Data_Generation"
data_generation_process = "DDR_Data_Generation"

In [3]:
num_train = 100 # number of training data
num_feat = 4 # size of feature
num_test = 10000
deg = 1.4 # polynomial degree
e = 1.0 # scale of normal std or the range of uniform. For the error term
num_arcs = 10

lower = 0 # coef lower bound
upper = 1 # coef upper bound
p = num_feat # num of features
d = num_arcs # num of arcs
alpha = e # scale of normal std or the range of uniform. For the error term
mis = deg # model misspecification
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 7

Oracle_mode = "Para"
Oracel_mode = "Realization"

In [4]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath = os.path.dirname(grandparent_directory) + '/Data/Shortest_Path_With_2_Nodes_0404_' + data_generation_process + "/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
print("DataPath:", DataPath)
DataPath = DataPath + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_x_dist="+x_dist+"_coef_seed="+str(coef_seed)+"_diff_W/"
pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_With_2_Nodes_0404_DDR_Data_Generation/


## Function to generate data

In [5]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}
    for iter in iteration_all:
        DataPath_seed = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        W_star = data_gen.generate_truth(DataPath_seed,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        thres = num_test
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], W_star_all[iter] = data_gen.generate_samples(iter,DataPath_seed,p, d, num_test, num_train, alpha, W_star, mis, thres, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 

        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, W_star_all

## Compute out-of-sample cost

In [6]:
def compute_oracle_cost(cost_test):
    data_size = np.shape(cost_test)[0]
    _cost = np.zeros(data_size)
    for j in range(data_size):
        _cost[j] = np.min(cost_test[j,:])
    return _cost

In [7]:
def compute_out_of_sample_cost(W,w0,cost_test,feature):
    data_size = np.shape(feature)[0]
    _cost = np.zeros(data_size)
    for j in range(data_size):
        cost = W @ feature[j,:] + w0
        opt_index = np.argmin(cost)
        cost_test_tem = cost_test[j,:]
        _cost[j] = cost_test_tem[opt_index]
    return _cost

# Oracle

In [8]:
iteration_all = np.arange(0,10)
x_test_all, c_test_all, x_train_all, c_train_all, W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)

In [9]:
cost_Oracle_para_all = {}; cost_Oracle_para_avg = np.zeros(len(iteration_all))
cost_Oracle_realization_all = {}; cost_Oracle_realization_avg = np.zeros(len(iteration_all))

for iter in iteration_all:
    cost_Oracle_realization_all[iter] = compute_oracle_cost(c_test_all[iter])
    cost_Oracle_realization_avg[iter] = np.nanmean(cost_Oracle_realization_all[iter])

    cost_Oracle_para_all[iter] = compute_out_of_sample_cost(W_star_all[iter],np.ones(num_arcs)*bump,c_test_all[iter],x_test_all[iter])
    cost_Oracle_para_avg[iter] = np.nanmean(cost_Oracle_para_all[iter])

## OLS

In [10]:
from OLS import ols_method
ols_method_obj = ols_method()
W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
cost_OLS_all = {}; cost_OLS_avg = np.zeros(len(iteration_all))
for iter in iteration_all:
    W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
    cost_OLS_all[iter] = compute_out_of_sample_cost(W_ols_all[iter],w0_ols_all[iter],c_test_all[iter],x_test_all[iter])
    cost_OLS_avg[iter] = np.nanmean(cost_OLS_all[iter])

Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13


# Prediction error

In [11]:
# iter = 1
# cost_OLS_pred = (W_ols_all[iter] @ x_train_all[iter].T).T + w0_ols_all[iter]
# cost_Oracle_pred = (W_star_all[iter] @ x_train_all[iter].T).T + np.ones(num_arcs) * bump

## DDR

In [12]:
def solve_DDR(mu,lamb,num_acrs,num_feat,num_train,c_input,x_input):
    model = ro.Model()            # create an RSOME model
    w0_d = model.dvar(num_acrs)
    W_d = model.dvar((num_acrs,num_feat))

    alpha = model.dvar(num_train)                   

    eps = model.dvar(num_train*num_acrs)                   
    eps_index = 0
    for i in range(num_train):
        for j in range(num_acrs):
            model.st(alpha[i] >= -mu * c_input[i,j] - (1 - mu) * (W_d[j,:] @ x_input[i,:] + w0_d[j]))
            model.st(eps[eps_index] == c_input[i,j] - W_d[j,:] @ x_input[i,:] - w0_d[j])
            eps_index = eps_index + 1
    model.min(rso.norm(eps) + sum(alpha) * (lamb/num_train)) 

    model.solve(cpt,display=False,log=False)
    # print("status=",model.solution.status)
    w0_d_sol = w0_d.get()
    W_d_sol = W_d.get()
    obj = model.obj()
    return w0_d_sol,W_d_sol,obj

In [13]:
def Loss(x_train, z_train, W, w0):
    N,p = x_train.shape
    N,d = z_train.shape
    a = []
    for n in range(N): # Number of samples
        for i in range(d): # Number of arcs
            temp = []
            for j in range(p): # Number of feature
                temp.append(x_train[n][j]*W[i,j])
            a.append((z_train[n][i] - sum(temp) - w0[i])*(z_train[n][i] - sum(temp) - w0[i]))      
    return np.sum(a)/N

In [14]:
def ddr_solver(x_train, z_train, thres, mu, lamb):
#     x_train = data[3]
    z_train_min = np.minimum(z_train,thres) 
    N,p = x_train.shape
    N,d = z_train.shape

    # DDR
    m = Model("ddr")
    #m.setParam("DualReductions",0)
    m.setParam('OutputFlag', 0)

    W_ind = tuplelist( [(i,j) for i in range(d) for j in range(p)] )
    w0_ind = tuplelist( [i for i in range(d)] )
    t_ind = tuplelist( [n for n in range(N)] )
    
    W_ddr = m.addVars(W_ind, lb=-GRB.INFINITY )
    w0_ddr = m.addVars(w0_ind, lb=-GRB.INFINITY )
    t_ddr = m.addVars( t_ind, lb=-GRB.INFINITY )

    m.setObjective( Loss(x_train, z_train, W_ddr, w0_ddr) + lamb*(quicksum([t_ddr[n]  for n in range(N)])/ N) , GRB.MINIMIZE)

    for n in range(N):
        for i in range(d):
            m.addConstr( (- mu*z_train[n,i] - (1- mu)*(quicksum([x_train[n][j]*W_ddr[i,j] for j in range(p)]) + w0_ddr[i])-\
                    t_ddr[n] <= 0))
    m.optimize()
    W = m.getAttr('x', W_ddr)
    w0 = m.getAttr('x', w0_ddr)
    t = m.getAttr('x', t_ddr)
    W_results = []
    for i in range(d):
        W_results.append([W[(i,j)] for j in range(p)])
    w0_results = [w0[i] for i in range(d)]
#     t_results = [t[n] for n in range(N)]
    return W_results, w0_results

In [15]:
mu_all = np.arange(0.1,1.0,0.1)
lamb_all = np.round(np.arange(0.0,1.0,0.1),4)

W_ddr_all = {}; w0_ddr_all = {}; obj_ddr_all = {}
cost_DDR_all = {}; cost_DDR_avg_all = {}
for iter in iteration_all:
    c_input = c_train_all[iter]
    x_input = x_train_all[iter]
    
    cost_ddr_avg_tem = np.zeros((len(mu_all),len(lamb_all)))
    for mu_index in range(len(mu_all)):
        mu = mu_all[mu_index]
        for lamb_index in range(len(lamb_all)):
            lamb = lamb_all[lamb_index]
            # print("iter = ",iter,",mu = ",mu,",lamb=",lamb)
            # w0_ddr_all[iter,mu,lamb],W_ddr_all[iter,mu,lamb],obj_ddr_all[iter,mu,lamb] = solve_DDR(mu,lamb,d,num_feat,num_train,c_input,x_input)
            W_ddr_all[iter,mu,lamb], w0_ddr_all[iter,mu,lamb] = ddr_solver(x_input, c_input, num_test, mu, lamb)
            cost_DDR_all[iter,mu,lamb] = compute_out_of_sample_cost(W_ddr_all[iter,mu,lamb],w0_ddr_all[iter,mu,lamb],c_test_all[iter],x_test_all[iter])
            cost_ddr_avg_tem[mu_index,lamb_index] = np.nanmean(cost_DDR_all[iter,mu,lamb])
    print("iter = ",iter,",ddr_vs_ols_ratio")
    print(np.round(cost_ddr_avg_tem/np.nanmean(cost_OLS_all[iter]),4))
    cost_DDR_avg_all[iter] = cost_ddr_avg_tem

iter =  0 ,ddr_vs_ols_ratio
[[1.     1.     1.     1.     1.     1.     1.     0.9999 1.     1.    ]
 [1.     1.     1.     0.9999 0.9999 1.     1.     0.9999 0.9999 0.9999]
 [1.     1.     1.     1.     1.     0.9999 0.9999 0.9999 0.9999 0.9999]
 [1.     1.     1.     1.     0.9999 1.     1.     1.     0.9999 0.9999]
 [1.     1.     1.     1.     0.9999 0.9999 0.9999 1.0001 1.0001 1.    ]
 [1.     1.     1.     1.     1.     0.9999 0.9999 0.9999 1.     1.    ]
 [1.     1.     1.     1.     1.     1.     1.     1.     0.9999 0.9999]
 [1.     1.     1.     1.     1.     1.     1.     1.     1.     1.    ]
 [1.     1.     1.     1.     1.     1.     1.     1.     1.     1.    ]]
iter =  1 ,ddr_vs_ols_ratio
[[1.     1.     1.     1.     1.     1.     1.     1.0001 1.0001 1.0001]
 [1.     1.     1.     1.     1.0001 1.     1.     1.0001 1.0001 1.0001]
 [1.     1.     1.     0.9999 1.     1.0001 1.0001 1.0001 1.0001 1.0001]
 [1.     1.     1.     1.     1.     1.     1.     1.     1.0001 1.

In [16]:
regret_DDR_vs_OLS_para_avg = np.zeros((len(mu_all),len(lamb_all)))
regret_DDR_vs_OLS_realization_avg = np.zeros((len(mu_all),len(lamb_all)))
for iter in iteration_all:
    regret_DDR_vs_OLS_para_avg = regret_DDR_vs_OLS_para_avg + (cost_OLS_avg[iter] - cost_DDR_avg_all[iter])/(cost_OLS_avg[iter] - cost_Oracle_para_avg[iter])
    regret_DDR_vs_OLS_realization_avg = regret_DDR_vs_OLS_realization_avg + (cost_OLS_avg[iter] - cost_DDR_avg_all[iter])/(cost_OLS_avg[iter] - cost_Oracle_realization_avg[iter])
regret_DDR_vs_OLS_para_avg = regret_DDR_vs_OLS_para_avg/len(iteration_all)
regret_DDR_vs_OLS_realization_avg = regret_DDR_vs_OLS_realization_avg/len(iteration_all)

In [18]:
regret_DDR_vs_OLS_para_avg_df = pd.DataFrame(regret_DDR_vs_OLS_para_avg)
regret_DDR_vs_OLS_para_avg_df.index = mu_all
regret_DDR_vs_OLS_para_avg_df.columns = lamb_all
regret_DDR_vs_OLS_para_avg_df.to_csv(DataPath+"regret_DDR_vs_OLS_para_avg.csv")

regret_DDR_vs_OLS_realization_avg = pd.DataFrame(regret_DDR_vs_OLS_realization_avg)
regret_DDR_vs_OLS_realization_avg.index = mu_all
regret_DDR_vs_OLS_realization_avg.columns = lamb_all
regret_DDR_vs_OLS_realization_avg.to_csv(DataPath+"regret_DDR_vs_OLS_realization_avg.csv")